Customizing Workflows
===================

This notebook shows how to write workflows for BQSKit, using the Hubbard Model$^1$ as an exemplar. The Hubbard Model is only briefly introduced; full details on the Hubbard Model, the Variational Quantum Eigensolver, the effects of noise, and more, can be found as part of the scikit-quant tutorials.$^2$

Prerequisites: OpenFermion, Qiskit, BQSKit, and SciPy (which will pull in NumPy), as well as optionally scikit-quant, e.g. install with pip:

`$ python -m pip install openfermion qiskit bqskit scipy`

Contents
-----------

This tutorial is setup to be run serially for the first several sections, as lessons learned (and code) are used in subsequent sections. However, afterwards it is possible to skip and jump ahead if a specific focus is of more interest.

* Work through in series:
   * [Introduction to the Hubbard Model](#Introduction)
   * [Effects of circuit structure on synthesis](#Energy-Objective)
   * [Running BQSKit synthesis in VQE](#Variational-Quantum-Eigensolver)
* Work through in series, or pick and choose:
   * [Customizing BQSKit workflows](#Customized-Compilation)
   * [Seeded synthesis](#Seeded-Synthesis)
   * [Implications of noise](#Implications-of-Noise)

Introduction
--------------

We will solve the Hubbard Model using the Variational Quantum Eigensolver (VQE) algorithm. The Hubbard Model (HM) has a range of uses, most notably in solid-state physics to explain phase transitions and other correlated behaviors. There are several variations of the HM, but in essence, the model describes interacting particles on a lattice. Despite its apparent simplicity, it is hard to solve on a classical computer because the known algorithms scale exponentially unless further approximations are made. In this tutorial, we will use the VQE algorithm to estimate the energy ground states of different configurations of an HM.

Basic Fermi-Hubbard Model
---------------------------------

We will restrict ourselves to a Hubbard Model with fermions (such as electrons), i.e. the Fermi-Hubbard Model. A visualization of an example with a 2x2 grid with spinful electrons is shown below. Electrons being fermions means that two electrons can not occupy the same quantum state. Because the electrons have spin, there can thus be at most two electrons per site (one spin-up, one spin-down). The model is characterized by an interaction term 'U', which only contributes if a spin-up and a spin-down electron occupy the same site; and by a "hopping" term 't', representing the kinetic energy in the system. In this basic model, only hops to neighboring sites are considered.

<table style="padding: 0px"><tr></tr><tr>
<td style="width: 65%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
The energy in the model is described by the following Hamiltonian using fermion algebra:
<br>
<p>\begin{equation*}
H = -t \sum_{ij,\sigma=\uparrow\downarrow} {a_{i\sigma}^\dagger a_{j\sigma}} + U \sum_i {a_{i\uparrow}^\dagger a_{i\uparrow} a_{i\downarrow}^\dagger a_{i\downarrow}}
\end{equation*}</p>
where:
<ul>
    <li>$a_{i\sigma}^\dagger$ = creation operator for site 'i' and spin $\sigma$</li>
    <li>$a_{i\sigma}$ = annihilation operator for site 'i' and spin $\sigma$</li>
    <li>U = interaction term</li>
    <li>t = hopping term</li>
</ul>
<p>The first sum quantifies moving electrons from each site 'j' to each neighboring site 'i' (first term, that is: $a_i^\dagger$ "creates" an electron at site 'i' while $a_j$ simultaneously "annihilates" it from site 'j'), and vice versa (second term), considering both up and down spins ($\sigma$) independently; the second sum quantifies the interaction contribution for having two electrons, one up and one down, at the same site 'i'.
</p></div></td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
<td style="width: 35%">
    <img src='./images/basic_fermi_hubbard_model.png'>
</td>
</tr></table>

<p>
Although the pictorial shows the electrons as concrete particles, the model treats them as waves that extend over all sites. A solution to the model provides the probabilities with which any of the electrons can be measured, as a particle, at each one of the sites.

Mapping fermions to qubits
---------------------------------

Fermion algebra is only a subset of what a quantum computer can express, since fermions are interchangeable and many states are thus completely equivalent, whereas qubits are identifiable (each one has a well defined location on a chip or ion trap, for example). Thus, it is useful to perform a mapping from fermion to qubit algebra that preserves the fermionic relations, to ensure that the mapped operators only produce states inside that fermionic subset; meaning that the computation should not, noise aside, produce non-physical states. Doing so reduces the scope for errors.$^3$

There are several possible mappings from fermion to qubit operators that preserve the fermion algebra. Here, we'll consider the Jordan-Wigner$^4$ and Bravyi-Kitaev$^5$ transforms, because these are commonly used and easy to understand and their differences$^6$ show an important point about how the optimizability of circuits.

To get started, first load some relevant modules and helper functions.

In [1]:
%load_ext autoreload

In [2]:
# useful tools
import logging
import numpy as np
import qiskit as qk
import scipy
from timeit import default_timer as timer

# BQSKit tooling
import bqskit
from bqskit.ext import qiskit_to_bqskit

# VQE objective function for Hamiltonians
from hubbard import EnergyObjective, hamiltonian_qiskit, hubbard

In [3]:
def display_circuit(circuit, basis=['u', 'cx']):
    """perform basic, unoptimized decomposition for circuit display"""

    uc =  qk.transpile(circuit, basis_gates=basis)
    
    for gate, count in uc.count_ops().items():
        print("gate %s count: %d" % (gate, count))

    return uc.draw()

The Fermi-Hubbard model is defined by the dimensions of the 2D grid, plus the hopping and interaction terms. In addition, OpenFermion allows some variations on the team by making the electrons spinless and making the grid periodic. Below is a very small model, just 2 sites (2x1), which will result in 2 classical optimization parameters. You can try a larger model, but this small example is chosen for a tutorial session where it can be easily run on a laptop on battery power.

In [4]:
# Setup for a 2x1 grid with a single electron.
hm_lattice = {
    'x_dimension' : 2,
    'y_dimension' : 1,      # 2D x_dim * y_dim grid
    'tunneling'   : 1.0,    # hpping term `t`
    'coulomb'     : 2.0,    # interaction term `U`
    'spinless'    : False,  # double negative, so do have spins
    'periodic'    : False,  # do not allow wrap-around hopping
}

hm_filling = {
    'n_electrons_up'   : 1,
    'n_electrons_down' : 0,
}

Energy Objective
--------------------

The Variational Quantum Eigensolver is an iterative algorithm where a classical optimizer minimizes an energy objective (the expectation value of the Hamiltonian) as calculated on the quantum computer. Typically, an Hamiltonian has many components, which can be evaluated in parallel, on separate quantum chips/computers. The final result is obtained by summing the parts classically using their proper weights.
<table style="padding: 0px"><tr></tr><tr>
<td style="width: 60%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
The trial quantum states proposed by the classical optimizer are constructed using an <i>Ansatz</i> that is parametrized using classical values only.
Here, we wil use <i>Unitary Coupled Cluster</i> Ansatz for single and double excitations (UCCSD). Briefly, this Ansatz explores the search space by moving single electrons and by swapping pairs.
This Ansatz is easier on the classical optimizer than many alternatives, but produces rather deep circuits, which is why the use of a tool like BQSKit becomes a necessity.
</div></td>
<td><img src='./images/vqe_structure.png'></td>
</tr></table>
To complete a circuit, we need to fill it with electrons and then instantiate the circuit for a set of optimization paremeters. Note that the meaning of these parameters depends on the chosen mapping.

In [5]:
# Hubbard Model Hamiltonian using Jordan-Wigner mapping
hm_jw = hamiltonian_qiskit(fermion_transform='jordan-wigner', **hm_lattice)

# create an energy objective, which can be minimized classically, from the mapped hamiltonian
jw_objective = EnergyObjective(hm_jw, **hm_filling)

# instantiate a circuit from a set of classical parameters
jw_circuit_instance = jw_objective.generate_circuit([0.1]*jw_objective.npar())

# visualize the circuit, using basic gates, and collect some statistics
c_jw = qk.transpile(jw_circuit_instance, basis_gates=["u3", "cx"])

for gate, count in c_jw.count_ops().items():
    print("gate %s count: %d" % (gate, count))
    
c_jw.draw()

gate cx count: 124
gate u3 count: 110


global phase: 2π
     ┌───────────────┐          ┌───┐┌────────────────┐┌───┐┌────────────────┐»
q_0: ┤ U3(π/2,-π,-π) ├──────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
     └┬─────────────┬┘     ┌───┐└─┬─┘└────────────────┘└─┬─┘└─────┬───┬──────┘»
q_1: ─┤ U3(π/2,0,π) ├──────┤ X ├──■──────────────────────■────────┤ X ├───────»
      ├─────────────┤ ┌───┐└─┬─┘                                  └─┬─┘       »
q_2: ─┤ U3(π/2,0,π) ├─┤ X ├──■──────────────────────────────────────■─────────»
     ┌┴─────────────┴┐└─┬─┘                                                   »
q_3: ┤ U3(π/2,0,π/2) ├──■─────────────────────────────────────────────────────»
     └───────────────┘                                                        »
«                                                     ┌───┐┌────────────────┐»
«q_0: ────────────────────────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├»
«     ┌────────────────┐                         ┌───┐└─┬─┘└────────────────┘»
«q_1: ┤ U3(0,π/2,-π/2) ├─────────────────────────┤ X ├──■────────────────────»
«     └─────┬───┬──────┘┌──────────────────┐┌───┐└─┬─┘                       »
«q_2: ──────┤ X ├───────┤ U3(π/2,π/2,-π/2) ├┤ X ├──■─────────────────────────»
«           └─┬─┘       ├──────────────────┤└─┬─┘                            »
«q_3: ────────■─────────┤ U3(π/2,-π/2,π/2) ├──■──────────────────────────────»
«                       └──────────────────┘                                 »
«     ┌───┐┌────────────────┐                                                  »
«q_0: ┤ X ├┤ U3(0,π/2,-π/2) ├──────────────────────────────────────────────────»
«     └─┬─┘└─────┬───┬──────┘┌──────────────────┐                         ┌───┐»
«q_1: ──■────────┤ X ├───────┤ U3(π/2,π/2,-π/2) ├─────────────────────────┤ X ├»
«                └─┬─┘       └──────┬───┬───────┘┌──────────────────┐┌───┐└─┬─┘»
«q_2: ─────────────■────────────────┤ X ├────────┤ U3(π/2,-π/2,π/2) ├┤ X ├──■──»
«                                   └─┬─┘        └┬────────────────┬┘└─┬─┘     »
«q_3: ────────────────────────────────■───────────┤ U3(0,π/2,-π/2) ├───■───────»
«                                                 └────────────────┘           »
«     ┌───┐┌───────────────┐┌───┐┌────────────────┐                  »
«q_0: ┤ X ├┤ U3(0,0,0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├──────────────────»
«     └─┬─┘└───────────────┘└─┬─┘└─────┬───┬──────┘┌────────────────┐»
«q_1: ──■─────────────────────■────────┤ X ├───────┤ U3(0,π/2,-π/2) ├»
«                                      └─┬─┘       └─────┬───┬──────┘»
«q_2: ───────────────────────────────────■───────────────┤ X ├───────»
«                                                        └─┬─┘       »
«q_3: ─────────────────────────────────────────────────────■─────────»
«                                                                    »
«                                   ┌───┐┌────────────────┐┌───┐»
«q_0: ──────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├»
«                              ┌───┐└─┬─┘└────────────────┘└─┬─┘»
«q_1: ─────────────────────────┤ X ├──■──────────────────────■──»
«     ┌──────────────────┐┌───┐└─┬─┘                            »
«q_2: ┤ U3(π/2,π/2,-π/2) ├┤ X ├──■──────────────────────────────»
«     ├──────────────────┤└─┬─┘                                 »
«q_3: ┤ U3(π/2,π/2,-π/2) ├──■───────────────────────────────────»
«     └──────────────────┘                                      »
«     ┌────────────────┐                                           ┌───┐»
«q_0: ┤ U3(0,π/2,-π/2) ├───────────────────────────────────────────┤ X ├»
«     └─────┬───┬──────┘┌────────────────┐                    ┌───┐└─┬─┘»
«q_1: ──────┤ X ├───────┤ U3(π/2,π/2,-π) ├────────────────────┤ X ├──■──»
«           └─┬─┘       └─────┬───┬──────┘ ┌────────────────┐ └─┬─┘     »
«q_2: ────────■───────────────┤ X ├────────┤ U3(0,π/2,-π/2) ├───■───────»
«                             └─┬─┘       ┌┴────────────────┴┐          »
«q_3: ──────────────────────────■─────────┤ U3(π/2,-π/2,π/2) ├──────────

**Exercise:** Instantiate an energy objective like before, but now for the Bravyi-Kitaev mapping. Draw and compare the generated circuits, find the main differences in structure, and try to guess how that might affect the synthesis, in particular based on what you have learned about partitioning.  

In [6]:
# Hubbard Model Hamiltonian using Bravyi-Kitaev mapping
hm_bk = hamiltonian_qiskit(fermion_transform='bravyi-kitaev', **hm_lattice)

# create an energy objective, which can be minimized classically, from the mapped hamiltonian
bk_objective = EnergyObjective(hm_bk, **hm_filling)

# instantiate a circuit from a set of classical parameters
bk_circuit_instance = bk_objective.generate_circuit([0.1]*bk_objective.npar())

# visualize the circuit, using basic gates, and collect some statistics
c_bk = qk.transpile(bk_circuit_instance, basis_gates=["u3", "cx"])

for gate, count in c_bk.count_ops().items():
    print("gate %s count: %d" % (gate, count))
    
c_bk.draw()

gate cx count: 90
gate u3 count: 80


global phase: 4.6124
      ┌───────────────┐      ┌───┐┌──────────────┐┌───┐┌────────────────┐»
q_0: ─┤ U3(π/2,-π,-π) ├──────┤ X ├┤ U3(0,0,0.05) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
     ┌┴───────────────┴┐┌───┐└─┬─┘└──────────────┘└─┬─┘└─────┬───┬──────┘»
q_1: ┤ U3(π/2,-π,-π/2) ├┤ X ├──■────────────────────■────────┤ X ├───────»
     └─┬─────────────┬─┘└─┬─┘                                └─┬─┘       »
q_2: ──┤ U3(π/2,0,π) ├────■────────────────────────────────────■─────────»
       └┬───────────┬┘                                                   »
q_3: ───┤ U3(π,0,π) ├────────────────────────────────────────────────────»
        └───────────┘                                                    »
«                              ┌───┐┌────────────────┐┌───┐┌────────────────┐»
«q_0: ─────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
«      ┌────────────────┐ ┌───┐└─┬─┘└────────────────┘└─┬─┘└─────┬───┬──────┘»
«q_1: ─┤ U3(π/2,π/2,-π) ├─┤ X ├──■──────────────────────■────────┤ X ├───────»
«     ┌┴────────────────┴┐└─┬─┘                                  └─┬─┘       »
«q_2: ┤ U3(π/2,π/2,-π/2) ├──■──────────────────────────────────────■─────────»
«     └──────────────────┘                                                   »
«q_3: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«                                 ┌───┐┌────────────────┐┌───┐»
«q_0: ────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├»
«                            ┌───┐└─┬─┘└────────────────┘└─┬─┘»
«q_1: ───────────────────────┤ X ├──■──────────────────────■──»
«     ┌────────────────┐┌───┐└─┬─┘                            »
«q_2: ┤ U3(0,π/2,-π/2) ├┤ X ├──■──────────────────────────────»
«     └────────────────┘└─┬─┘                                 »
«q_3: ────────────────────■───────────────────────────────────»
«                                                             »
«     ┌────────────────┐                                   ┌───┐»
«q_0: ┤ U3(0,π/2,-π/2) ├───────────────────────────────────┤ X ├»
«     └─────┬───┬──────┘┌───────────────┐                  └─┬─┘»
«q_1: ──────┤ X ├───────┤ U3(π/2,0,π/2) ├────────────────────┼──»
«           └─┬─┘       └─────┬───┬─────┘┌────────────────┐  │  »
«q_2: ────────■───────────────┤ X ├──────┤ U3(0,π/2,-π/2) ├──■──»
«                             └─┬─┘      └────────────────┘     »
«q_3: ──────────────────────────■───────────────────────────────»
«                                                               »
«     ┌────────────────┐┌───┐┌────────────────┐     ┌───┐┌────────────────┐»
«q_0: ┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├─────┤ X ├┤ U3(0,0,-0.025) ├»
«     └────────────────┘└─┬─┘└────────────────┘     └─┬─┘└────────────────┘»
«q_1: ────────────────────┼───────────────────────────┼────────────────────»
«                         │  ┌────────────────┐┌───┐  │                    »
«q_2: ────────────────────■──┤ U3(0,π/2,-π/2) ├┤ X ├──■────────────────────»
«                            └────────────────┘└─┬─┘                       »
«q_3: ───────────────────────────────────────────■─────────────────────────»
«                                                                          »
«     ┌───┐┌──────────────────┐                       ┌───┐┌──────────────┐»
«q_0: ┤ X ├┤ U3(π/2,π/2,-π/2) ├───────────────────────┤ X ├┤ U3(0,0,0.05) ├»
«     └─┬─┘└──────────────────┘                  ┌───┐└─┬─┘└──────────────┘»
«q_1: ──┼────────────────────────────────────────┤ X ├──■──────────────────»
«       │         ┌───┐        ┌────────────────┐└─┬─┘                     »
«q_2: ──■─────────┤ X ├────────┤ U3(0,π/2,-π/2) ├──■───────────────────────»
«                 └─┬─┘        └────────────────┘                          »
«q_3: ──────────────■──────────────────────────────────────────────────────»
«                                                                          »
«     ┌───┐┌────────────────┐        

Effect of Fermion to Qubit Mapping
-----------------------------------------

The mapping has to encode the positions of the electrons and do so with a guarantee of anti-symmetry of the fermionic wave functions, to respect fermion algebra. There are thus two pieces of information: the electron occupancy on each site and the overall parity. The Jordan-Wigner mapping fully localizes the occupation number (each qubit represents the occupation on a site), which then causes the parity to be completely non-localized: whenever an electron "hops" sites, the parity update requires an update to all qubits, propagated through a chain of controlled gates. The Bravyi-Kitaev transform encodes both occupancy and parity information non-locally, leading to a more balanced result.

**Exercise:** Check the circuits again and find that the Jordan-Wigner mapping results in many more series of CNOTs involving all qubits than does Bravyi-Kitaev. These series of CNOTs across all qubits are a worst-case scenario for synthesis, as they prevent further partitioning and are already optimal.

<div class="alert alert-block alert-info">
<b>Note:</b> the circuits above were generated using the same number of Trotter steps (a method of approximating an exponential) and although the Bravyi-Kitaev circuit is significantly shorter in that case, the approximation is actually slightly worse. After adjusting the number of steps and the method to achieve the same quality of approximation (use <tt>trotter_steps=2, trotter_mode='trotter'</tt> for Jordan-Wigner and <tt>trotter_steps=3, trotter_mode='trotter'</tt>), both methods are much closer in the number of CNOTs in the unoptimized circuit. Since Trotter steps are just repetitions, this does not change any conclusions about the relative effectiveness of BQSKit's circuit synthesis on the respective mappings.
</div>

**Exercise:** Use what you learned in the __[BQSKit introductory notebook](1_comping_with_bqskit.ipynb)__ to compile both circuit instances. Try for `optimization_level=2` and `max_synthesis_size=3` first, adjust and play with the parameters if time allows (do note that larger block size and/or higher optimization level can result in significantly longer compilation times). Compare and explain the results.

**Optional exercise:** The examples so far have presumed all-to-all. Experiment with one of the realistic QPU models, or your own. Are the results and conclusions the same?

In [7]:
circuits = ((c_jw, 'Jordan-Wigner'), (c_bk, 'Bravyi-Kitaev'))

for circuit, mapping in circuits:
    opt_circuit = bqskit.compile(qiskit_to_bqskit(circuit), optimization_level=2, max_synthesis_size=3)

    # Print new circuit statistics
    print("Compiled Circuit Statistics for:", mapping)
    print("  Gate Counts:", opt_circuit.gate_counts)
    print("  Connectivity:", opt_circuit.coupling_graph)

Compiled Circuit Statistics for: Jordan-Wigner
  Gate Counts: {CNOTGate: 108, U3Gate: 117}
  Connectivity: CouplingGraph({(2, 3), (1, 2), (0, 1)})
Compiled Circuit Statistics for: Bravyi-Kitaev
  Gate Counts: {CNOTGate: 77, U3Gate: 65}
  Connectivity: CouplingGraph({(0, 1), (1, 2), (2, 3), (0, 2), (1, 3)})


Variational Quantum Eigensolver
--------------------------------------

In the VQE algorithm, circuits are synthesized based on parameters provided by a classical minimizer. All we need to do, is to pass the energy objective function to a classical optimizer. Since we are not considering noise here (see the [scikit-quant tutorials](https://github.com/scikit-quant/scikit-quant/tree/master/tutorials), if interested), most any optimizer will do. Here, we will use [BFGS from SciPy](https://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize).

In [8]:
min_energy = scipy.optimize.minimize(jw_objective, [0.1]*jw_objective.npar(), method='BFGS')
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_jw, **hm_filling)))

minimum energy: -1.00000 (exact: -1.00000)


**Exercise:** Verify the results for the objective using the Bravyi-Kitaev mapping. Optionally, try different fillings and grid sizes (note that larger grids will have local minima where the optimizer can get stuck if not seeded close to the global minimum).

In [9]:
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

minimum energy: -1.00000 (exact: -1.00000)


The Ansatz forms the bulk of any VQE circuit: both the input state and the Hamiltonian components only require single qubit rotations, at most, at the beginning and at the very end to measure the desired projection components for each qubit.
It thus makes sense to optimize the Ansatz part of the circuit independently and re-use it (with at most a fast peep-hole pass to merge single qubit gates at start and end where possible).

**Exercise:** The `EnergyObjective` is written such that a derived class can implement the `bqskit` optimization. Fill in the remainder below, and verify the results. Use the added pass to collect statistics on the `bqskit` results.

In [10]:
from hubbard import EnergyObjective
from bqskit.compiler.compiler import Compiler

# to show BQSKit is running
log = logging.getLogger("bqskit.compiler")
log.setLevel(logging.INFO)

class MyObjective(EnergyObjective):
    def optimize_evolution(self, bqskit_circuit):
        return bqskit.compile(
            bqskit_circuit,
            optimization_level=2,
            max_synthesis_size=3,
        )
    
bk_objective = MyObjective(hm_bk, **hm_filling)
start = timer()
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("converged in %.0f seconds" % (timer() - start))
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

(8642993664): 14:57:15.840 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8642993664): 14:57:15.841 - INFO     | bqskit.compiler.compiler: Compiling task: a3336222-130f-4912-98dc-ad28fcb9ffe4


INFO:bqskit.compiler.compiler:Compiling task: a3336222-130f-4912-98dc-ad28fcb9ffe4


(8642993664): 14:57:15.850 - INFO     | bqskit.compiler.compiler: Submitted task: a3336222-130f-4912-98dc-ad28fcb9ffe4


...

(8642993664): 15:05:04.594 - INFO     | bqskit.compiler.compiler: Submitted task: 275903d6-11b0-4f1a-aeab-e1401b723b42


INFO:bqskit.compiler.compiler:Submitted task: 275903d6-11b0-4f1a-aeab-e1401b723b42


(8642993664): 15:05:08.758 - INFO     | bqskit.compiler.compiler: Stopped compiler process.


INFO:bqskit.compiler.compiler:Stopped compiler process.


converged in 476 seconds
minimum energy: -1.00000 (exact: -1.00000)


## Customized Compilation
-----------------------------

It is often useful to customize the compilation workflow that is setup by the `compile()` function, either because there are specific BQSKit tools you want to use, because you want to fine-tune settings, or simply because you want to add diagnostics to better understand the synthesis performance.

A compilation workflow consists of one or more `CompilationTask`s that are constructed from a list of passes that are executed in order.
The available passes are listed in the [documentation](https://bqskit.readthedocs.io/en/latest/source/passes.html). There are passes for partitioning, synthesis, conversions, iterative, conditionals, checkpointing, etc. etc. Of course, you can also write your own custom passes.

Once you have a `CompilationTask`, you can use the BQSKIt compiler to compile it:

```
from bqskit.compiler.compiler import Compiler
from bqskit.compiler.task import CompilationTask

circuit = ...
workflow = ...

task = CompilationTask(circuit, workflow)
with Compiler() as compiler:
    opt_circuit = compiler.compile(task)
```

**Exercise:** The `compile()` function uses the LEAP algorithm by default. Below are some example workflows to get started trying different partitioners and algorithms. Make sure to look up each pass in the [documentation](https://bqskit.readthedocs.io/en/latest/source/passes.html), and don't hesitate to try your own mix. Try them either with one of the earlier single circuits, or implement them in the `MyObjective` function from the previous exercise to use with a complete VQE run.

```
from bqskit import MachineModel
from bqskit.passes.mapping.setmodel import SetModelPass
from bqskit.passes import *

workflow = [
    SetModelPass(model),
    QuickPartitioner(),
    ForEachBlockPass([
        QSearchSynthesisPass(),
        ScanningGateRemovalPass()
    ]),
    UnfoldPass()
]
```

**Exercise:** To improve your understanding, here are two possible diagnostic passes that can be inserted into the workflow to show/debug the effect of invidual passes. Add them in your workflow as needed and/or create your own:

```
from bqskit.compiler.basepass import BasePass

class PrintCNOTsPass(BasePass):
    def run(self, circuit, data) -> None:
        print("BQSKit step, current CNOT count:", circuit.count(bqskit.ir.gates.CNOTGate()))
        
class PrintPartitionInfoPass(BasePass):
    def run(self, circuit, data = {}):
        acm = 0
        for op in circuit:            
            if not isinstance(op.gate, CircuitGate):
                print("Circuit not yet partitioned")
                return

            acm += op.gate._circuit.num_operations

        print("Number of partitions:", circuit.num_operations)
        print("Average block size:", acm / circuit.num_operations)
```

<div class="alert alert-block alert-info">
<b>Note:</b> the printing passes will be run in a Dask process and jupyter will therefore not pick up the output. You will find the printed output in the terminal where jupyter was started.
</div>

In [11]:
from bqskit.compiler.compiler import Compiler
from bqskit.compiler.task import CompilationTask
from bqskit.ir.gates import CircuitGate
from bqskit import MachineModel
from bqskit.compiler.basepass import BasePass
from bqskit.passes.mapping.setmodel import SetModelPass
from bqskit.passes import *

class PrintCNOTsPass(BasePass):
    def run(self, circuit, data) -> None:
        print("BQSKit step, current CNOT count:", circuit.count(bqskit.ir.gates.CNOTGate()))

class PrintPartitionInfoPass(BasePass):
    def run(self, circuit, data = {}):
        acm = 0
        for op in circuit:            
            if not isinstance(op.gate, CircuitGate):
                print("Circuit not yet partitioned")
                return

            acm += op.gate._circuit.num_operations

        print("Number of partitions:", circuit.num_operations)
        print("Average block size:", acm / circuit.num_operations)

coupling_graph = [
    (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)
]
model = MachineModel(4, coupling_graph=coupling_graph)

workflow = [
    PrintCNOTsPass(),
    SetModelPass(model),
    PrintPartitionInfoPass(),
    QuickPartitioner(),
    PrintPartitionInfoPass(),
    ForEachBlockPass([
        QSearchSynthesisPass(),
        ScanningGateRemovalPass()
    ]),
    UnfoldPass(),
    PrintCNOTsPass()
]

task = CompilationTask(qiskit_to_bqskit(c_bk), workflow)
with Compiler() as compiler:
    opt_circuit = compiler.compile(task)
    
print("Gate Counts:", opt_circuit.gate_counts)
print("Connectivity:", opt_circuit.coupling_graph)

(8642993664): 15:05:11.858 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8642993664): 15:05:11.860 - INFO     | bqskit.compiler.compiler: Compiling task: 7878fb60-c372-4b63-8356-6d14409a58f7


INFO:bqskit.compiler.compiler:Compiling task: 7878fb60-c372-4b63-8356-6d14409a58f7


(8642993664): 15:05:11.869 - INFO     | bqskit.compiler.compiler: Submitted task: 7878fb60-c372-4b63-8356-6d14409a58f7


INFO:bqskit.compiler.compiler:Submitted task: 7878fb60-c372-4b63-8356-6d14409a58f7


(8642993664): 15:05:20.942 - INFO     | bqskit.compiler.compiler: Stopped compiler process.


INFO:bqskit.compiler.compiler:Stopped compiler process.


Gate Counts: {CNOTGate: 65, U3Gate: 125}
Connectivity: CouplingGraph({(0, 1), (1, 2), (0, 3), (2, 3), (0, 2), (1, 3)})


Seeded Synthesis
---------------------

A circuit optimized by BQSKit is no longer explicitly parametrized (optimizing a fully instantiate circuit leads to better results than when the parametrization is left in-place in the form of gates), but can still serve as a template. Many classical optimizers use a parameter search (e.g. gradient descent), where the next iteration is a tweak on the current parameter set, meaning that the current circuit instance requested is likely to be close to the previous one. Furthermore, even when the optimizer uses a different approach, such as a stencil or grid, where the parameters are further apart, the overall structure of the final optimized current circuit is still likely to be close to the previous one, because it still describes the same physical system.

BQSKit can make use of this observation with so-called "seeded" synthesis: use the previous instantiation as a template to synthesize the next circuit.

**Exercise:** Store the result circuit of the current iteration in a data member of the objective. Then next round, use its `instantiate()` method to generate a new circuit. Verify whether the new circuit instantiated from the old template is close enough using a threshold of your choosing. Explain the instantiation behavior. Verify the result from minimization.

In [12]:
from timeit import default_timer as timer

class MyObjective(EnergyObjective):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.template = None

    def optimize_evolution(self, bqskit_circuit):
        opt_circuit = None

        if self.template is not None:
            target_unitary = bqskit_circuit.get_unitary()
            opt_circuit = self.template.instantiate(target_unitary)    
            if 1e-8 < self.template.get_unitary().get_distance_from(target_unitary, 1):
                opt_circuit = None    # reject this circuit
            else:
                print("accepting instantiation from template")
        
        if opt_circuit is None:
            opt_circuit = bqskit.compile(
                bqskit_circuit,
                optimization_level=2,
                max_synthesis_size=3,
            )

        self.template = opt_circuit

        return opt_circuit
 
bk_objective = MyObjective(hm_bk, **hm_filling)
start = timer()
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("converged in %.0f seconds" % (timer() - start))
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

(8642993664): 15:05:24.219 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8642993664): 15:05:24.220 - INFO     | bqskit.compiler.compiler: Compiling task: f852d353-f713-4304-9454-50011f5c3ff7


INFO:bqskit.compiler.compiler:Compiling task: f852d353-f713-4304-9454-50011f5c3ff7


(8642993664): 15:05:24.229 - INFO     | bqskit.compiler.compiler: Submitted task: f852d353-f713-4304-9454-50011f5c3ff7


...

(8642993664): 15:07:10.993 - INFO     | bqskit.compiler.compiler: Submitted task: 24da6fe3-f295-4a95-8db0-b0517c5aca77


INFO:bqskit.compiler.compiler:Submitted task: 24da6fe3-f295-4a95-8db0-b0517c5aca77


(8642993664): 15:07:15.222 - INFO     | bqskit.compiler.compiler: Stopped compiler process.


INFO:bqskit.compiler.compiler:Stopped compiler process.


accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
accepting instantiation from template
converged in 122 seconds
minimum energy: -1.00000 (exact: -1.00000)


<div class="alert alert-block alert-info">
<b>Note:</b> instantation from a template is fundamental to BQSKit and it can be added directly to passes, for example as an argument to any of the synthesis passes to seed the synthesis from, see the <a href="https://bqskit.readthedocs.io/en/latest/source/autogen/bqskit.passes.SeedLayerGenerator.html">SeedLayerGenerator</a>.
</div>

Implications of Noise
-------------------------

The use of BQSKit truly comes into its own when there is noise present in the system, as is the case in any current, real, QPU: the circuit optimization will show real improvements in the final result. As a note of caution, however: when noise is present, several things change. For example, a noise-resilient optimizer is needed and even then, the final energy result obtained is likely to be higher than the global minimum, even when the final parameters are spot-on. Additionally, simulation with noise takes considerably longer than without, and multiple runs are needed for convincing statistics.

Install scikit-quant, to make its optimizers available as a Qiskit optimizer:

    `$ python -m pip install scikit-quant`

Construct a simple noise model (the circuits as-is will, even after optimization, be too long to be run on current hardware; you can try that by setting `noise_model='realistic'`) and pass it through the constructor of the `EnergyObjective`. Run with the base `EnergyObjective` and with your derived implementation using BQSKit. Compare the results with and without `bqskit` optimization.

In [13]:
from qiskit.algorithms.optimizers import IMFIL
from hubbard import noise_model

logging.getLogger('hubbard').setLevel(logging.INFO)

In [14]:
noise = noise_model.create(p_gate_1q=0.01, p_gate_2q=0.02)

In [15]:
objective = EnergyObjective(hm_bk, noise_model=noise, **hm_filling)
npar = objective.npar()

result = IMFIL(maxiter=100).minimize(objective, np.array([0.1]*npar), bounds=np.array([(-1,1)]*npar))
print("minimum energy: %.5f (at solution %s: %.5f; exact: %.5f)" % (
    result.fun, result.x, EnergyObjective(hm_bk, **hm_filling)(result.x), hubbard.exact(hm_bk, **hm_filling)))

INFO:hubbard:objective: 0.97980 @ [0.1 0.1]
INFO:hubbard:objective: 0.86255 @ [-0.9  0.1]
INFO:hubbard:objective: 0.97405 @ [ 0.1 -0.9]
INFO:hubbard:objective: 0.90974 @ [-0.29886665  0.08046573]
INFO:hubbard:objective: 1.05076 @ [0.70113335 0.08046573]
INFO:hubbard:objective: 0.90972 @ [-0.29886665 -0.91953427]
INFO:hubbard:objective: 0.86401 @ [-0.778621   0.0804244]
INFO:hubbard:objective: 1.00109 @ [0.221379  0.0804244]
INFO:hubbard:objective: 0.86508 @ [-0.778621  -0.9195756]
INFO:hubbard:objective: 0.89522 @ [-0.4  0.1]
INFO:hubbard:objective: 0.86311 @ [-0.9  0.6]
INFO:hubbard:objective: 0.86238 @ [-0.9 -0.4]
INFO:hubbard:objective: 0.86449 @ [-1.          0.09751847]
INFO:hubbard:objective: 0.86450 @ [-1.          0.09875923]
INFO:hubbard:objective: 0.86329 @ [-0.95556229  0.09937962]
INFO:hubbard:objective: 0.86282 @ [-0.92778114  0.09968981]
INFO:hubbard:objective: 0.89522 @ [-0.4  0.1]
INFO:hubbard:objective: 0.86311 @ [-0.9  0.6]
INFO:hubbard:objective: 0.86238 @ [-0.9 -0.4

minimum energy: 0.86232 (at solution [-0.89786741 -0.23566251]: -0.97481; exact: -1.00000)


In [16]:
objective = MyObjective(hm_bk, noise_model=noise, **hm_filling)
npar = objective.npar()

result = IMFIL(maxiter=100).minimize(objective, np.array([0.1]*npar), bounds=np.array([(-1,1)]*npar))
print("minimum energy: %.5f (at solution %s: %.5f; exact: %.5f)" % (
    result.fun, result.x, EnergyObjective(hm_bk, **hm_filling)(result.x), hubbard.exact(hm_bk, **hm_filling)))

(8642993664): 15:07:40.015 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8642993664): 15:07:40.016 - INFO     | bqskit.compiler.compiler: Compiling task: 3fd140d0-6647-43ad-ad1c-6ab985b3e569


INFO:bqskit.compiler.compiler:Compiling task: 3fd140d0-6647-43ad-ad1c-6ab985b3e569


(8642993664): 15:07:40.026 - INFO     | bqskit.compiler.compiler: Submitted task: 3fd140d0-6647-43ad-ad1c-6ab985b3e569


...

accepting instantiation from template


INFO:hubbard:objective: 0.98217 @ [-0.9078125  0.475    ]


accepting instantiation from template


INFO:hubbard:objective: 0.98555 @ [-0.9        0.4671875]
INFO:hubbard:objective: -0.97385 @ [-0.9    0.475]


minimum energy: 0.96089 (at solution [-0.9    0.475]: -0.97385; exact: -1.00000)


## References
-------------

[1] J. Hubbard, Proc. Roy. Soc. (Lond.) 276 (1963) 238
<br>[2] https://github.com/scikit-quant/scikit-quant/tree/master/tutorials
<br>[3] https://arxiv.org/abs/cond-mat/0012334, Ortiz et al., "Quantum Algorithms for Fermionic Simulations"
<br>[4] P. Jordan and E. Wigner, "Über das Paulische Äquivalenzverbot", Z. Phys. 47, 631 (1928)
<br>[5] https://arxiv.org/abs/quant-ph/0003137 S. Bravyi, A. Kitaev, "Fermionic quantum computation"
<br>[6] https://arxiv.org/abs/1812.02233, A. Tranter, et al., "A comparison of the Bravyi-Kitaev and Jordan-Wigner transformations for the quantum simulation of quantum chemistry"